In [ ]:
import socket
from time import time,sleep
import json
import threading
import base64

In [ ]:
class Connection:
    def __init__(self, HOST='127.0.0.1', PORT=10240,debug=False):
        """
        初始化套接字，绑定指定端口
        """
        # socket.gethostname()
        self.BUFFSIZE = 102400
        self.ADDR = (HOST, PORT)
        self.s = None
        self.c = None
        self.s = socket.socket(family=socket.AF_INET, type=socket.SOCK_STREAM)
        #self.s.setsockopt(socket.IPPROTO_TCP, socket.TCP_NODELAY, 1)
        #self.s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        self.s.bind(self.ADDR)
        self.s.listen(5)
    def __del__(self,):
        if self.s is not None:
            self.s.close()
        print('close socket')
    def wait_conn(self):
        """
        等待连接
        """
        print("等待连接ing")
        self.c, self.addr = self.s.accept()        
        print('Got connection from', self.c, self.addr)
    def wait_conn2(self,name):
        """
        等待连接
        """
        print("等待连接ing  %s"%name)
        while True:
            self.c, self.addr = self.s.accept()        
            print('Got connection from', self.addr, name)
            sleep(1)
    def read_buf(self,):
        """
        读取缓存区，返回最新一次的str
        若插件未运行，则会报错
        """
        try:
            #if debug:
            #print('network try receive')
            data_bytes = self.c.recv(self.BUFFSIZE)
            #print('network receive success')
        except Exception as e:
            print(e)
            print('scoket receive error!')
            self.close_conn()
        data_str = data_bytes.decode(encoding="utf-8")
        #start_pos = data_str.rfind('[')
        #data_str = data_str[start_pos:]
        return data_str
    def write_buf(self, commands):
        print('network send')
        self.c.sendall((str(commands) + '\0').encode(encoding="utf-8"))
    def close_conn(self,):
        """
        关闭连接
        """
        if self.c is not None:
            self.c.close()
            self.c = None
            # print("close connect")net

In [ ]:
send_flag = True
cache_buf = []
localhost='10.212.47.98'
#localhost='127.0.0.1'
gamehost='10.212.47.201'
eye_conn = Connection(localhost,PORT=10240)
hand_conn = Connection(localhost,PORT=10241)
head_conn = Connection(localhost,PORT=10242)
send_conn = Connection(PORT=10243)

#眼-手-脑-游戏
#hand_conn.wait_conn()
threading.Thread(target=hand_conn.wait_conn2,args=('hand',)).start()

#head_conn.wait_conn()
threading.Thread(target=head_conn.wait_conn2,args=('head',)).start()
#eye_conn.wait_conn()
threading.Thread(target=eye_conn.wait_conn2,args=('eye',)).start()
send_conn.wait_conn()


time_threshold = 5

hand_flag = False
eye_flag = False

eye,hand,head = '[]','[]','[]'

def clear_all():
    global eye,hand,head
    eye,hand,head = '[]','[]','[]'
    
def receive_eye():
    
    global eye
    while True:
        eye =  eye_conn.read_buf()
        if len(eye)>0:
            print(eye)
            
            
def receive_hand():
    global hand
    while True:
        hand = hand_conn.read_buf()
        if len(hand)>0:
            print(hand)
            hand_buf = [hand[:]]
            s=hand_buf[0][-2:-1]
            d=['','护卫','警戒','攻击','围攻']
            try:
                send_conn.c.sendall(base64.b64encode(bytes('ui:%s'%d[int(s)], encoding='utf8'))+b'\n')
            except:
                pass

        
def receive_head():
    global head
    while True:
        head = head_conn.read_buf()
        if len(head)>0:
            print(head)
        #if 'head' in head:
        #print(head)



threading.Thread(target=receive_head,).start()
threading.Thread(target=receive_hand,).start()
threading.Thread(target=receive_eye,).start()


#send_conn.setsockopt(socket.SOL_TCP, socket.TCP_NODELAY, 1)
#host = '10.180.76.202'  # socket.gethostname()
#send_conn.connect((host, 5134))

In [ ]:
eye_count=0
clear_all()
while True:
    sleep(0.1)
    hand_buf = [hand[:]]
    cache_buf = [eye[:]]
    if 'hand_index' in hand_buf[0]:#手势指令可以打断眼动指令
        print("收到手势一阶指令，进入手势指令处理逻辑")
        eye_flag = False
        clear_all()
        hand_flag = True#这个标记在超时和手势指令处理完毕时候置为False
        hand_start = time()
        while(True):#收到手势信号后，眼动信号不可打断手势处理
            hand_buf2 = [hand[:]]
            if 'hand_index' in hand_buf2[0]:
                hand_buf=hand_buf2.copy()
            if 'head' in head:
                send_cmd = hand_buf[0][-2:-1]
                print("收到手势确认指令，发送指令至平台",send_cmd)
                if send_flag:
                    send_conn.c.sendall(base64.b64encode(bytes(send_cmd, encoding='utf8'))+b'\n')
                break
            if time() - hand_start > time_threshold:
                print("手势确认超时")
                break
        hand_buf = []
        clear_all()
        hand_flag = False
    elif 'eye_index' in cache_buf[0] and hand_flag is False:#不存在手势指令时才收到眼动指令
        print("收到眼动一阶指令")
        clear_all()
        c = cache_buf.copy()
        send_cmd = 'light:'+c[0][c[0].rfind(':')+1:-1] 
        send_conn.c.sendall(base64.b64encode(bytes(send_cmd, encoding='utf8'))+b'\n')
        eye_flag = True
        start = time()
    elif eye_flag and 'head' in head:
        print("进入眼动二阶处理逻辑")
        eye_start = time()
        clear_all()
        eye_buf = []
        while(True):
            #if 'eye_direction' in eye:
            #    print("收到眼动二阶指令")
            #    eye_buf = [eye]
            #    clear_all()
            #el
            eye_buf=['eye_direction:a']
            if 'head' in head and len(eye_buf)!=0:#给出了方向，且进行确认
                send_cmd = c[0][c[0].rfind(':')+1:-1] + ':a' + eye_buf[0][eye_buf[0].rfind(':')+1:-1]
                if send_flag and eye_count<3:
                    print("收到眼动确认指令，发送指令至平台",send_cmd)
                    send_conn.c.sendall(base64.b64encode(bytes(send_cmd, encoding='utf8'))+b'\n')
                    eye_count+=1
                break
            if time() - eye_start > time_threshold+8:# 5秒内需做出指定方向的指令
                print("眼动二阶处理超时")
                break
        clear_all()
        eye_flag = False
    elif 'eye_direction' in eye:
        print('diret')
    else:
        clear_all()
    if eye_flag  and time() - start > 8:
        eye_flag = False
        print("由于长时间未收到确认指令，清空一阶指令")
    clear_all()
    #    pass